## ⚖️ Quick Decision Guide: LangChain vs LangGraph

### 🔗 Use **LangChain** when:
- You know the exact sequence of steps needed and tasks are executed in a specific order  
- Building simple chatbots, Q&A systems, or RAG pipelines  
- You need quick prototyping  
- Your workflow is linear like:  
```

Load PDF → chunk → embed → retrieve → answer

```

---

### 📈 Use **LangGraph** when:
- You need stateful, multi-agent systems with complex, nonlinear workflows that involve loops and revisiting previous states  
- Your logic must adapt, e.g., ask clarifying questions if confidence is low, retry on failures, or pause for human input  
- Building autonomous agents that make decisions  
- You need to maintain context over long interactions  


---
📢 Discover more Agentic AI notebooks on my [GitHub repository](https://github.com/lisekarimi/agentverse) and explore additional AI projects on my [portfolio](https://lisekarimi.com).

## 💻 Code Examples

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from typing import TypedDict, Literal
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display


### 💬 Example 1: Simple Q&A System

#### 🔗 LangChain Version (Best for this use case)

In [ ]:
# Simple linear chain: prompt → LLM → parse output
llm = ChatOpenAI(model="gpt-4")
prompt = ChatPromptTemplate.from_template("Answer this question: {question}")
output_parser = StrOutputParser()

# Create a chain using LangChain Expression Language (LCEL)
chain = prompt | llm | output_parser

# Execute
result = chain.invoke({"question": "What is the capital of France?"})
print(result)

Why LangChain? This is a straightforward, one-directional flow. No loops, no complex state, just input → process → output.

#### 📈 LangGraph Version (Overkill for this, but here's how it looks)

In [ ]:
class State(TypedDict):
    question: str
    answer: str

def ask_llm(state: State) -> State:
    # Call LLM here
    state["answer"] = "Paris"  # Simplified
    return state

# Build graph
graph = StateGraph(State)
graph.add_node("ask_llm", ask_llm)
graph.add_edge(START, "ask_llm")
graph.add_edge("ask_llm", END)

app = graph.compile()
result = app.invoke({"question": "What is the capital of France?"})
print(result["answer"])

# Generate and display the graph
graph_image = app.get_graph().draw_mermaid_png()
display(Image(graph_image))

### 🎯 Example 2: Customer Support Agent with Decision Making

#### 🔗 LangChain Version (Gets messy)

In [ ]:
# Define tools
def check_order_status(order_id: str) -> str:
    return f"Order {order_id} is shipped"

def process_refund(order_id: str) -> str:
    return f"Refund initiated for {order_id}"

# Create agent - but controlling flow is harder
llm = ChatOpenAI(model="gpt-4")
tools = [check_order_status, process_refund]

# LangChain agents can use tools, but:
# - Hard to add conditional logic (if X, do Y, else Z)
# - Can't easily loop back to ask clarifying questions
# - State management requires extra work

#### 📈 LangGraph Version (Much better for this!)

In [ ]:
# Define tools (same as LangChain example)
def process_refund(order_id: str) -> str:
    return f"Refund initiated for {order_id}"

class State(TypedDict):
    messages: list[str]
    user_intent: str
    order_id: str
    confidence: float
    needs_clarification: bool
    clarification_count: int  # Add counter to prevent infinite loops

def classify_intent(state: State) -> State:
    # Analyze user message
    # ... LLM call to classify (would use state["messages"][-1])
    last_message = state["messages"][-1] if state["messages"] else ""

    # Check if we already have an order_id (from clarification response)
    if state.get("order_id") and state["order_id"]:
        # If we have order_id, we're confident enough to proceed
        state["user_intent"] = "refund"
        state["confidence"] = 0.9
    elif "refund" in last_message.lower():
        state["user_intent"] = "refund"
        # Low confidence if no order_id provided
        state["confidence"] = 0.6 if not state.get("order_id") else 0.9
    else:
        state["user_intent"] = "unknown"
        state["confidence"] = 0.3

    return state

def should_clarify(state: State) -> Literal["clarify", "execute"]:
    # Conditional routing based on confidence
    # Also check if we've asked too many times
    if state.get("clarification_count", 0) >= 2:
        # After 2 clarifications, proceed anyway
        return "execute"
    if state["confidence"] < 0.7:
        return "clarify"
    return "execute"

def ask_clarification(state: State) -> State:
    state["messages"].append("Which order would you like to refund?")
    state["clarification_count"] = state.get("clarification_count", 0) + 1
    # Simulate getting order_id from user response (in real app, this would come from user input)
    # For demo purposes, set a default order_id after asking
    if state["clarification_count"] == 1:
        # Simulate: user responds with order_id
        state["order_id"] = "ORD-12345"
    return state

def execute_action(state: State) -> State:
    if state["user_intent"] == "refund":
        order_id = state.get("order_id") or "UNKNOWN"
        result = process_refund(order_id)
        state["messages"].append(result)
    return state

# Build the graph with conditional logic
graph = StateGraph(State)

graph.add_node("classify", classify_intent)
graph.add_node("clarify", ask_clarification)
graph.add_node("execute", execute_action)

graph.add_edge(START, "classify")
graph.add_conditional_edges(
    "classify",
    should_clarify,  # Function decides next node
    {"clarify": "clarify", "execute": "execute"}
)
graph.add_edge("clarify", "classify")  # Loop back!
graph.add_edge("execute", END)

app = graph.compile()

# Execute with state persistence
result = app.invoke({
    "messages": ["I want a refund"],
    "user_intent": "",
    "order_id": "",
    "confidence": 0.0,
    "needs_clarification": False,
    "clarification_count": 0
})

print("Final state:")
for msg in result["messages"]:
    print(f"  - {msg}")
print(f"\nIntent: {result['user_intent']}")
print(f"Order ID: {result.get('order_id', 'N/A')}")
print(f"Confidence: {result['confidence']}")

# Generate and display the graph
graph_image = app.get_graph().draw_mermaid_png()
display(Image(graph_image))


Why LangGraph?

- The graph structure allows for loops and revisiting previous states, making it ideal for interactive systems Medium
- Can pause and ask for clarification
- Maintains conversation context across steps
- Clear, debuggable flow

### 🔬 Example 3: Multi-Agent Research System

#### 📈 LangGraph Version (Perfect use case)

In [ ]:
class ResearchState(TypedDict):
    topic: str
    web_results: list[str]
    academic_papers: list[str]
    summary: str
    quality_score: float

def web_researcher(state: ResearchState) -> ResearchState:
    # Agent 1: Search the web
    state["web_results"] = ["result1", "result2"]
    return state

def academic_researcher(state: ResearchState) -> ResearchState:
    # Agent 2: Search academic databases
    state["academic_papers"] = ["paper1", "paper2"]
    return state

def synthesizer(state: ResearchState) -> ResearchState:
    # Agent 3: Combine findings
    all_info = state["web_results"] + state["academic_papers"]
    state["summary"] = f"Summary of {len(all_info)} sources"
    state["quality_score"] = 0.85
    return state

def quality_check(state: ResearchState) -> str: # Routing function
    # Decide if we need more research
    if state["quality_score"] < 0.8:
        return "web"  # Loop back for more data
    return "done"

# Build multi-agent graph
graph = StateGraph(ResearchState)

# Add nodes
graph.add_node("web", web_researcher)
graph.add_node("academic", academic_researcher)
graph.add_node("synthesize", synthesizer)

# Add simple edges
graph.add_edge(START, "web")
graph.add_edge("web", "academic")
graph.add_edge("academic", "synthesize")

# Add conditional edge (the loop!)
graph.add_conditional_edges(
    "synthesize",
    quality_check,
    {"web": "web", "done": END}
)

# Compile
app = graph.compile()

# Run it
result = app.invoke({
    "topic": "AI Research",
    "web_results": [],
    "academic_papers": [],
    "summary": "",
    "quality_score": 0.0
})

print(result)

# Generate and display the graph
graph_image = app.get_graph().draw_mermaid_png()
display(Image(graph_image))


Why LangGraph? Multiple agents need to coordinate, share state, and potentially loop back for more information.